# Topic clustering using LDA

### 1. Clean data and return tokens.

In [1]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


###  Find the meanings of words, synonyms,antonyms using WordNet

In [2]:
import nltk
nltk.download('wordnet')



[nltk_data] Downloading package wordnet to
[nltk_data]     /home/quicksilver/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)


In [4]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/quicksilver/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Prepare text for topic clustering

In [5]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens


In [6]:
import random
text_data = []
with open('bigtranscript.txt') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        #print(tokens)
        text_data.append(tokens)

* First, we are creating a dictionary from the data, then converting to a bag-of-words corpus and saving the dictionary and corpus for future use.
* We are using Gensim for this particular task.

In [7]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)


In [8]:
corpus = [dictionary.doc2bow(text) for text in text_data]

### At present, we are limiting number of topics per mixture to 5

In [9]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30)
ldamodel.save('model21.gensim')

In [10]:


topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)



(0, '0.022*"think" + 0.018*"people" + 0.011*"would" + 0.011*"company"')
(1, '0.027*"designer" + 0.016*"actually" + 0.012*"things" + 0.011*"think"')
(2, '0.025*"design" + 0.017*"going" + 0.016*"people" + 0.014*"right"')
(3, '0.017*"things" + 0.014*"think" + 0.013*"designer" + 0.012*"client"')
(4, '0.032*"think" + 0.010*"things" + 0.009*"always" + 0.008*"people"')


We have been able to generate top 5 topic mixtures from our collection of transcripts.

### Further Work

* Try lda2vec and compare results with lda.
* Assign labels to topics generated by the unsupervised learning model.